<a href="https://colab.research.google.com/github/rlawogjs96/deeplearningnlp_starters/blob/main/Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Cosine Similarity
코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미한다. 두 벡터의 방향이 완전히 동일하면 1의 값, 90도 각을 이루면 0, 180도 반대의 방향을 가지면 -1의 값을 갖게 된다. 두 벡터가 가리키는 방향이 얼마나 유사한가를 의미한다. 문서 단어 행렬이나 TF-IDF 행렬을 통해서 문서의 유사도를 구하는 경우 문서 단어 행렬이나 TF-IDF 행렬이 각각의 특징 벡터 A,B가 된다. 

* 문서1 : 저는 사과 좋아요
* 문서2 : 저는 바나나 좋아요
* 문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

In [7]:
import numpy as np
from numpy import dot 
from numpy.linalg import norm

In [8]:
def cos_sim(A,B): 
  return dot(A,B)/(norm(A)*norm(B))

In [9]:
doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서 2의 유사도: ',cos_sim(doc1,doc2))
print('문서 1과 문서 3의 유사도: ',cos_sim(doc1,doc3))
print('문서 2와 문서 3의 유사도: ',cos_sim(doc2,doc3))

문서 1과 문서 2의 유사도:  0.6666666666666667
문서 1과 문서 3의 유사도:  0.6666666666666667
문서 2와 문서 3의 유사도:  1.0000000000000002


#### 2. 유사도를 이용한 추천 시스템 구현하기
Kaggle Movie Dataset을 갖고 영화 추천시스템을 구현. TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템 개발.

In [10]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [12]:
!pwd
%cd ./gdrive/MyDrive/archive

/content
/content/gdrive/MyDrive/archive


In [13]:
data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


Cosine Similarity에 사용할 데이터는 영화제목에 해당하는 title 열과 줄거리에 해당하는 overview 열이다. 좋아하는 영화를 입력하면, 해당 영화의 줄거리와 유사한 줄거리의 영화를 찾아서 추천하는 시스템을 개발.

In [16]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)

In [17]:
# 데이터에 Null 값이 들어가있으면 에러 발생. TF-IDF의 대상이 되는 data의 overview 열에 결측값에 해당하는 Null값이 있는지 확인.
print('overview 열의 결측값의 수: ',data['overview'].isnull().sum())

overview 열의 결측값의 수:  135


In [18]:
# 결측값을 빈 값으로 대체 
data['overview'] = data['overview'].fillna('')

In [19]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape): ', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape):  (20000, 47487)


20000개의 영화를 표현하기 위해 총 47487개의 단어가 사용되었다. 47847차원의 문서 벡터가 20000개가 존재한다. 

In [21]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('Result of Cosine Similarity: ', cosine_sim.shape)

Result of Cosine Similarity:  (20000, 20000)


20000개의 각 문서 벡터 (영화 줄거리 벡터)와 자기 자신을 포함한 20000개의 문서 벡터간의 유사도가 기록된 행렬. 모든 20000개의 영화의 상호 유사도가 기록되어있다. 기존 데이터프레임으로부터 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index를 만든다.

In [23]:
title_to_index = dict(zip(data['title'], data.index)) 

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴 
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [24]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

In [25]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object